In [2]:
#!pip install siphon metpy

In [3]:
# Import necessary libraries
from datetime import datetime
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir

In [ ]:

dt = datetime(2024, 9, 30, 12)
station = 'BMX'
df = WyomingUpperAir.request_data(dt, station)
display(df)
p = df['pressure'].values
t = df['temperature'].values
u = df['u_wind'].values
v = df['v_wind'].values

print(p)


In [15]:
from datetime import datetime
import pandas as pd
from siphon.simplewebservice.wyoming import WyomingUpperAir

# Define the date and time for which you want the data
dt = datetime(2024, 10, 1, 12)  # Change the date as per your requirement

# List of stations
stations = {'Del Rio, Texas': '72261', 'Omaha, Nebraska': '72558', 'International Falls, Minnesota': '72747'}

# Function to filter data between 925 hPa and 500 hPa and export to CSV
def export_filtered_data(df, location):
    # Filter the dataframe for pressures between 925 hPa and 500 hPa
    layer_df = df[(df['pressure'] <= 925) & (df['pressure'] >= 500)]
    # Export the filtered data to a CSV file
    filename = f"{location.replace(',', '').replace(' ', '_')}_pressure_vs_temperature.csv"
    pressure_and_temperature.to_csv(filename, index=False)
    return filename, layer_df

# Function to calculate mean temperature using the pressure-weighted formula
def calculate_weighted_mean_temperature(layer_df):
    # Calculate weighted mean temperature
    weighted_sum = (layer_df['pressure'] * layer_df['temperature']).sum()
    pressure_sum = layer_df['pressure'].sum()
    weighted_mean_temp = weighted_sum / pressure_sum if pressure_sum != 0 else None
    return weighted_mean_temp

# Loop through the stations and retrieve data
for location, station in stations.items():
    # Request data
    df = WyomingUpperAir.request_data(dt, station)
    
    # Export filtered data to CSV and get the filename and filtered dataframe
    filename, filtered_df = export_filtered_data(df, location)
    
    # Calculate the pressure-weighted mean temperature
    weighted_mean_temp = calculate_weighted_mean_temperature(filtered_df)
    
    # Print the mean temperature
    if weighted_mean_temp is not None:
        print(f"{location}: Weighted Mean Temperature (925-500 hPa) = {weighted_mean_temp:.2f}°C")
    else:
        print(f"{location}: Unable to compute weighted mean temperature due to insufficient data.")


HTTPError: Error accessing http://weather.uwyo.edu/cgi-bin/sounding/?region=naconf&TYPE=TEXT%3ALIST&YEAR=2024&MONTH=10&FROM=0112&TO=0112&STNM=72261
Server Error (503: Sorry, the server is too busy to process your request.
Please try again later.
)